# Data Quality with DuckGuard - 10x Faster Than Pandas

**DuckGuard** is a Python-native data quality tool built on DuckDB.

In this notebook, we'll use DuckGuard to:
1. Profile the Titanic dataset
2. Get instant quality scores
3. Detect semantic types and PII
4. Create validation rules
5. Detect anomalies

[![GitHub](https://img.shields.io/badge/GitHub-XDataHubAI%2Fduckguard-blue)](https://github.com/XDataHubAI/duckguard)
[![PyPI](https://img.shields.io/pypi/v/duckguard.svg)](https://pypi.org/project/duckguard/)

In [ ]:
# Install DuckGuard
!pip install duckguard -q
print("DuckGuard installed!")

In [ ]:
# Connect to Kaggle dataset (adjust path as needed)
# For Titanic competition:
# data = connect("/kaggle/input/titanic/train.csv")
# For this demo, we'll create sample data
import pandas as pd

from duckguard import connect, detect_anomalies, execute_rules, load_rules_from_string
from duckguard.semantic import SemanticAnalyzer

df = pd.DataFrame({
    'PassengerId': range(1, 101),
    'Survived': [0, 1] * 50,
    'Pclass': [1, 2, 3, 1, 2] * 20,
    'Name': [f'Passenger {i}' for i in range(1, 101)],
    'Sex': ['male', 'female'] * 50,
    'Age': [25, 30, None, 45, 22] * 20,
    'Fare': [50.0, 75.5, 10.0, 200.0, 15.5] * 20,
    'Embarked': ['S', 'C', 'Q', 'S', None] * 20
})
df.to_csv('titanic_sample.csv', index=False)

data = connect('titanic_sample.csv')
print(f"Loaded {data.row_count} rows, {data.column_count} columns")

## 1. Instant Quality Score

In [ ]:
quality = data.score()

print("=" * 50)
print("DATA QUALITY REPORT")
print("=" * 50)
print(f"\nOverall Score: {quality.overall:.1f}/100")
print(f"Grade: {quality.grade}")
print("\nDimension Scores:")
print(f"  Completeness: {quality.completeness:.1f}")
print(f"  Uniqueness: {quality.uniqueness:.1f}")
print(f"  Validity: {quality.validity:.1f}")
print(f"  Consistency: {quality.consistency:.1f}")

## 2. Column Statistics

In [ ]:
# Check for missing values
print("Missing Values:")
for col in data.columns:
    null_pct = getattr(data, col).null_percent
    if null_pct > 0:
        print(f"  {col}: {null_pct:.1f}% null")

print("\nFare Statistics:")
print(f"  Min: {data.Fare.min}")
print(f"  Max: {data.Fare.max}")
print(f"  Mean: {data.Fare.mean:.2f}")

## 3. Semantic Type Detection

In [ ]:
from duckguard import detect_types_for_dataset

types = detect_types_for_dataset(data)
print("Detected Semantic Types:")
for col, sem_type in types.items():
    type_name = sem_type.value if sem_type else "generic"
    print(f"  {col}: {type_name}")

# Check for PII
analyzer = SemanticAnalyzer()
analysis = analyzer.analyze(data)
if analysis.pii_columns:
    print(f"\n⚠️ Potential PII in: {analysis.pii_columns}")
else:
    print("\n✓ No obvious PII detected")

## 4. YAML Validation Rules

In [ ]:
rules_yaml = """
dataset: titanic
rules:
  - PassengerId is not null
  - PassengerId is unique
  - Survived in [0, 1]
  - Pclass in [1, 2, 3]
  - Age >= 0
  - Fare >= 0
  - Sex in ['male', 'female']
"""

rules = load_rules_from_string(rules_yaml)
result = execute_rules(rules, dataset=data)

print(f"Results: {result.passed_count}/{result.total_checks} passed")
print(f"Quality Score: {result.quality_score:.1f}%")
print("\nDetails:")
for r in result.results:
    status = "✓ PASS" if r.passed else "✗ FAIL"
    print(f"  [{status}] {r.check.expression}")

## 5. Anomaly Detection

In [ ]:
report = detect_anomalies(data, method="iqr", threshold=1.5)

print("Anomaly Detection (IQR method):")
print(f"Columns checked: {report.statistics.get('columns_checked', 0)}")
print(f"Anomalies found: {report.anomaly_count}")

for a in report.anomalies:
    status = "⚠️ ANOMALY" if a.is_anomaly else "✓ OK"
    print(f"  {status} {a.column}: {a.message}")

## 6. Auto-Generate Rules

In [ ]:
from duckguard import generate_rules

# Auto-generate rules from data
generated = generate_rules(data, dataset_name="titanic")
print("Auto-Generated Rules:")
print(generated)

## Summary

DuckGuard provides:
- **Instant quality scores** with A-F grades
- **Semantic type detection** including PII
- **YAML-based validation rules**
- **Anomaly detection** with multiple methods
- **10x faster** than pandas-based tools

### Install & Learn More
```bash
pip install duckguard
```

- GitHub: https://github.com/XDataHubAI/duckguard
- PyPI: https://pypi.org/project/duckguard/

In [ ]:
print("=" * 70)
print("DUCKGUARD 3.0 - TITANIC DATASET VALIDATION SUITE")
print("=" * 70)

validation_results = {}

# 1. Basic Quality Checks
print("\n[1/4] Basic Quality Checks...")
validation_results['unique_ids'] = data.PassengerId.is_unique()
validation_results['valid_survival'] = data.Survived.isin([0, 1])
print(f"  ✓ Unique IDs: {validation_results['unique_ids'].passed}")
print(f"  ✓ Valid survival values: {validation_results['valid_survival'].passed}")

# 2. Conditional Validation
print("\n[2/4] Conditional Validation...")
validation_results['fare_range'] = data.Fare.between_when(
    min_value=50.0,
    max_value=1000.0,
    condition="Pclass = 1"
)
print(f"  ✓ First class fare range: {validation_results['fare_range'].passed}")

# 3. Multi-Column Relationships
print("\n[3/4] Multi-Column Relationships...")
validation_results['pclass_valid'] = data.expect_column_pair_satisfy(
    column_a="Pclass",
    column_b="Pclass",
    expression="Pclass IN (1, 2, 3)",
    threshold=1.0
)
print(f"  ✓ Valid passenger class: {validation_results['pclass_valid'].passed}")

# 4. Query-Based Business Logic
print("\n[4/4] Query-Based Business Logic...")
validation_results['no_dups'] = data.expect_query_to_return_no_rows(
    query="SELECT PassengerId, COUNT(*) FROM table GROUP BY PassengerId HAVING COUNT(*) > 1"
)
validation_results['survival_rate'] = data.expect_query_result_to_be_between(
    query="SELECT AVG(Survived) * 100 FROM table",
    min_value=20.0,
    max_value=60.0
)
print(f"  ✓ No duplicates: {validation_results['no_dups'].passed}")
print(f"  ✓ Survival rate in range: {validation_results['survival_rate'].passed}")

# Summary
print("\n" + "=" * 70)
passed = sum(1 for r in validation_results.values() if r.passed)
total = len(validation_results)
quality_score = (passed / total) * 100

print(f"VALIDATION COMPLETE: {passed}/{total} checks passed ({quality_score:.0f}%)")
print("=" * 70)

if quality_score == 100:
    print("\n🎉 Perfect! Dataset is ready for ML modeling.")
elif quality_score >= 80:
    print("\n✅ Good! Minor issues detected, but dataset is usable.")
else:
    print("\n⚠️ Warning! Significant data quality issues detected.")

## 🎯 Complete 3.0 Validation Pipeline

Combine all features for production-grade data validation:

In [ ]:
# Distributional tests require scipy
!pip install scipy -q

try:
    # Test if Age follows normal distribution
    result = data.Age.expect_distribution_normal(significance_level=0.05)

    print(f"Age follows normal distribution: {result.passed}")
    print(f"P-value: {result.details.get('pvalue', 0):.4f}")
    print(f"Mean age: {result.details.get('mean', 0):.1f}")
    print(f"Std dev: {result.details.get('std', 0):.1f}")

    # Test if Fare distribution matches expected
    result2 = data.Fare.expect_ks_test(distribution='expon')
    print(f"\nFare follows exponential: {result2.passed}")

    # Chi-square test for categorical distribution
    # Test if passenger class distribution is as expected
    result3 = data.Pclass.expect_chi_square_test(
        expected_frequencies={1: 0.25, 2: 0.25, 3: 0.50},  # Expected proportions
        significance_level=0.05
    )

    print(f"\nClass distribution matches expected: {result3.passed}")
    print(f"P-value: {result3.details.get('pvalue', 0):.4f}")

except ImportError:
    print("scipy not available for distributional tests")

## 🆕 New in 3.0: Distributional Testing

Perfect for ML feature validation - test if your features follow expected distributions:

In [ ]:
# Check 1: No duplicate passenger IDs
result = data.expect_query_to_return_no_rows(
    query="""
        SELECT PassengerId, COUNT(*) as cnt 
        FROM table 
        GROUP BY PassengerId 
        HAVING cnt > 1
    """
)

print(f"No duplicates: {result.passed}")

# Check 2: Verify we have both survivors and non-survivors
result2 = data.expect_query_to_return_rows(
    query="SELECT * FROM table WHERE Survived = 1"
)

print(f"Have survivors: {result2.passed}")

# Check 3: Check survival rate is in reasonable range
result3 = data.expect_query_result_to_be_between(
    query="SELECT AVG(Survived) * 100 FROM table",
    min_value=20.0,  # At least 20% survived
    max_value=60.0   # At most 60% survived
)

print(f"Survival rate in range: {result3.passed}")
print(f"Actual rate: {result3.actual_value:.1f}%")

## 🆕 New in 3.0: Query-Based Checks

Write custom SQL to validate business logic that's hard to express otherwise:

In [ ]:
# Business rule: Survivors (Survived=1) should be recorded
result = data.expect_column_pair_satisfy(
    column_a="Survived",
    column_b="PassengerId",
    expression="Survived IN (0, 1)",
    threshold=1.0  # 100% must satisfy
)

print(f"Survival data validity: {result.passed}")
print(f"Message: {result.message}")

# More complex: Validate Pclass and Fare relationship
# Higher class (lower Pclass number) generally means higher fare
result2 = data.expect_column_pair_satisfy(
    column_a="Pclass",
    column_b="Fare",
    expression="(Pclass = 3 AND Fare < 50) OR (Pclass IN (1, 2))",
    threshold=0.8  # Allow 20% exceptions
)

print(f"\nClass-Fare relationship: {result2.passed}")
print(f"Satisfaction rate: {result2.details.get('satisfaction_rate', 0):.1%}")

## 🆕 New in 3.0: Multi-Column Validation

Validate complex relationships between columns using SQL expressions:

In [ ]:
# Example: Age should not be null for adult passengers (Age >= 18)
# (Some children may have missing ages, but adults should have them)
result = data.Age.not_null_when("Age >= 18 OR Age IS NULL")

print(f"Conditional check: {result.passed}")
print(f"Details: {result.message}")

# Example 2: First class passengers (Pclass=1) should have cabin assignments
# result2 = data.Cabin.not_null_when("Pclass = 1")
# print(f"\nCabin check: {result2.passed}")

# Example 3: Fare validation based on class
result3 = data.Fare.between_when(
    min_value=50.0,
    max_value=1000.0,
    condition="Pclass = 1"
)

print(f"\nFirst class fare range: {result3.passed}")
print(f"Violations: {result3.details.get('violations', 0)}")

## 🆕 New in 3.0: Conditional Expectations

DuckGuard 3.0 adds powerful conditional validation - check values only when conditions are met: